In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
import gc

from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.metrics import log_loss
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import os
from scipy.sparse import coo_matrix
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
train = pd.read_pickle('../ffm_cache/online_train_raw')

In [6]:
test = train.iloc[10602212:13625807,:]
train = train.iloc[0:10602212,:]

In [3]:
COLUMNS = ['age_cut','gender', 'education', 'marriageStatus', 'haveBaby','appID','creativeID']

In [3]:
cate_features = ['creativeID', 'userID', 'positionID', 'connectionType',
       'telecomsOperator', 'age', 'gender', 'education', 'marriageStatus',
       'haveBaby', 'ht_province', 'rd_province', 'sitesetID', 'positionType',
       'adID', 'camgaignID', 'advertiserID', 'appID', 'appPlatform',
       'appCategory', 'trick', 'clickHour', 'first_diff', 'last_diff',
       'install2click', 'posSum_online', 'creSum_online', 'userSum_online',
       'adSum_online', 'appSum_online', 'positionID_cvr_smooth',
       'creativeID_cvr', 'userID_cvr', 'adID_cvr', 'appID_cvr',
       'user_hist_install', 'installed_cate_0', 'installed_cate_1',
       'installed_cate_2', 'installed_cate_3', 'installed_cate_4',
       'installed_cate_5', 'user_ad_click_day', 'user_camgaign_click_day',
       'user_camgaign_click_hour', 'user_app_click_day', 'user_app_click_hour',
       'user_site_click_day', 'user_site_click_hour', 'user_click_day',
       'user_ad_click_day_min', 'user_camgaign_click_day_min',
       'user_app_click_day_max', 'user_app_click_day_min',
       'user_site_click_day_max', 'user_site_click_day_min',
       'user_click_day_max', 'user_click_day_min', 'user_ad_click_day_mean',
       'user_app_click_day_mean', 'user_site_click_day_mean',
       'user_click_day_mean']

In [5]:
lbe = LabelEncoder()
id_size_dict ={}
for feature in tqdm(cate_features):
    train[feature] = lbe.fit_transform(train[feature])
    id_size_dict[feature] = train[feature].max()+1

100%|██████████| 62/62 [01:31<00:00,  1.32s/it]


In [6]:
test_x = train.iloc[2901508:5925103,:]
train_x = train.iloc[0:2901508,:]

In [7]:
del train
gc.collect()

96

In [8]:
id_size_list = []

In [9]:
for feature in cate_features:
    id_size_list.append(id_size_dict[feature])

In [10]:
def run(fetches,sess, X=None, y=None, ):
        feed_dict = {}
        if type(X) is list:
            for i in range(len(X)):
                feed_dict[id_inputs[i]] = X[i]
                #print(feed_dict[id_inputs[i]])
        else:
            feed_dict[id_inputs] = X
        if y is not None:
            feed_dict[label_inputs] = y
        return sess.run(fetches, feed_dict)

In [14]:
input_feature = cate_features#['appID','gender','connectionType']
embedding_size =8
id_dtype= tf.int32
id_field_num = len(input_feature)
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(0)
    global_step =  tf.Variable(0,dtype=tf.int32,trainable=False,name='global_step')

    with tf.name_scope('ID_input'):
        id_inputs =[tf.placeholder(id_dtype,shape=[None,]) for i in range(id_field_num)]
        label_inputs = tf.placeholder(tf.float32)
    
    with tf.name_scope('Embedding_matrix'):
        embeddings = [tf.Variable(tf.ones([id_size_list[i],embedding_size],)) for i in range(0,id_field_num)]
        #embedding1 = tf.Variable(tf.ones([id_size,embedding_size],))
        #embedding2 = tf.Variable(tf.ones([id_size,embedding_size],))
        #embedding3 = tf.Variable(tf.ones([id_size,embedding_size],))
        onehot_embeddings = [tf.Variable(tf.random_normal([id_size_list[i],1])) for i in range(0,id_field_num)]
        #onehot_embedding1 = tf.Variable(tf.random_normal([id_size,1]))
        #onehot_embedding2 = tf.Variable(tf.random_normal([id_size,1]))
        #onehot_embedding3 = tf.Variable(tf.random_normal([id_size,1]))
    
    with tf.name_scope('Embedding_layer'):
        embeds = [tf.nn.embedding_lookup(embeddings[i],id_inputs[i]) for i in range(0,id_field_num)]
        #embed1 = tf.nn.embedding_lookup(embedding1,appID)
        #embed2 = tf.nn.embedding_lookup(embedding2,genderID)
        #embed3 = tf.nn.embedding_lookup(embedding3,connectionTypeID)
        
        onehot_embeds = [tf.nn.embedding_lookup(onehot_embeddings[i],id_inputs[i]) for i in range(0,id_field_num)]
        #onehot_embed1 = tf.nn.embedding_lookup(onehot_embedding1,appID)
        #onehot_embed2 = tf.nn.embedding_lookup(onehot_embedding2,genderID)
        #onehot_embed3 = tf.nn.embedding_lookup(onehot_embedding3,connectionTypeID)
    
    with tf.name_scope('FM_layer'):
        fm_units = []
        embed_list=embeds#[embed1,embed2,embed3]
        for i in range(0,id_field_num):
            for j in range(i+1,id_field_num):
                temp = tf.reduce_sum(tf.multiply(embed_list[i],embed_list[j]),axis=1)
                fm_units.append(temp)

        sum_fm_units =tf.reduce_sum(fm_units,0)
        single_onehot_list = tf.concat(onehot_embeds,1)
        sum_linear_units = tf.reduce_sum(single_onehot_list,1)
    with tf.name_scope('Output'):
        logits = tf.add(sum_fm_units,sum_linear_units)
        prob = tf.sigmoid(logits)
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=label_inputs,logits=logits))
        batch_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=label_inputs,logits=logits))
        optimizer = tf.train.AdamOptimizer().minimize(loss,global_step=global_step)
        #optimizer = tf.train.FtrlOptimizer(0.05).minimize(loss,global_step=global_step)
    with tf.name_scope('Summary'):
        tf.summary.scalar('loss', loss)
        summary_op = tf.summary.merge_all()

In [ ]:
input_feature = cate_features
early_stop_round = 20
min_loss = 1e9
chance_round = 0

In [ ]:
train_size = train_x.shape[0]
valid_size = test_x.shape[0]
batch_size = 1024#10240
valid_batch_size = 10240
batch_num = train_size//batch_size
valid_batch_num = valid_size//valid_batch_size
num_round = 1000
va_input_X = [test_x[feature].values for feature in input_feature]
va_input_y = test_x['label'].values

with tf.Session(graph=graph) as sess:
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter('./deepFM_log/',graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    for i in range(num_round):
        
        for j in tqdm(range(0,1+batch_num)): 
            X_i = train_x.iloc[j*batch_size:min((j+1)*batch_size,train_size)]
            input_X = [X_i[feature].values for feature in input_feature]#X_i[input_feature]
            input_y =X_i['label'].values
            _,l,summary = run([optimizer,loss,summary_op],sess=sess,X = input_X,y = input_y)

        va_loss_sum = 0 
        for j in range(0,1+valid_batch_num):
            X_i = test_x.iloc[j*valid_batch_size:min((j+1)*valid_batch_size,valid_size)]
            input_X = [X_i[feature].values for feature in input_feature]#X_i[input_feature]
            input_y =X_i['label'].values
            va_batch_loss = run([batch_loss,],sess=sess,X=input_X,y=input_y,)
            va_loss_sum += va_batch_loss[0]
        
        va_loss = va_loss_sum/valid_size
        print('epoch {0} va-loss {1:.7f}'.format(i,va_loss))
        if va_loss < min_loss:
            min_loss = va_loss
            chance_round = 0
        else:
            chance_round +=1
        if va_loss < 0.104 or chance_round > early_stop_round:
            print('done')
            saver.save(sess,'./LR_log/save_model',global_step=global_step)
            break
    
        saver.save(sess,'./FM_log/save_model',global_step=global_step)       

In [58]:
va_loss

2.9091137032515957